In [1]:
import subprocess
import statistics
import re
from math import sqrt

def benchmark_classes(classes, num_iters, ref_means, ref_errs):
    percent_means = []
    percent_err = []

    for i, class_ in enumerate(classes):
        percent_mean, percent_err = benchmark_class(class_, num_iters, ref_means[i], ref_errs[i])
        percent_means.append(percent_mean)
        percent_err.append(percent_err)

    return percent_means, percent_err


def benchmark_class(class_, num_iters, ref_mean, ref_err):
    runtime_mean, runtime_err = execute_class(class_, num_iters)
    percent_means, percent_err = compare_to_ref(runtime_mean, runtime_err, ref_mean, ref_err)

    return percent_means, percent_err


def execute_classes(classes, num_iters):
    runtime_means = []
    runtime_errs = []

    for i, class_ in enumerate(classes):
        runtime_mean, runtime_err = execute_class(class_, num_iters)
        runtime_means.append(runtime_mean)
        runtime_errs.append(runtime_err)

    return runtime_means, runtime_errs


def execute_class(class_, num_iters):
    cmd = f'./bin/ft.{class_}'

    runtimes = []

    print(f'Class: {class_}, Iterations: {num_iters}')

    for i in range(num_iters):
        res = subprocess.check_output(cmd.split(' ')).decode('utf-8')

        match = re.search(r'Verification\s+=\s+(\w+)', res)
        assert match, 'No match found!'
        assert match.group(1) == 'SUCCESSFUL', f'Verification = {match.group(1)}'

        match = re.search(r' Time in seconds\s+=\s+(\d+\.\d+)', res)
        assert match, 'No match found!'
        runtime = float(match.group(1))
        runtimes.append(runtime)

        runtime_mean, runtime_err = calc_stats(runtimes)
        print(f'  [{(i+1):03d}] Runtime: {runtime_mean:>8.6f} ± {runtime_err:.6f} [sec] ({runtime:>8.6f})')

    print()

    return runtime_mean, runtime_err


def compare_to_ref(runtime_mean, runtime_err, ref_mean, ref_err):
    diff_mean = ref_mean - runtime_mean
    diff_err = sqrt(ref_err**2 + runtime_err**2)

    percent_mean = diff_mean / ref_mean * 100.0
    percent_err = sqrt((diff_err/diff_mean)**2 + (ref_err/ref_mean)**2) * abs(percent_mean) if diff_mean != 0.0 else 0.0

    print(f'Runtime: {runtime_mean:.6f} ± {runtime_err:.6f} [sec]')
    print(f'Ref    : {ref_mean:.6f} ± {ref_err:.6f} [sec]')
    print(f'Diff   : {diff_mean:.6f} ± {diff_err:.6f} [sec]')
    print(f'Speedup: {percent_mean:>7.2f}% ± {percent_err:>7.2f}%')

    return percent_mean, percent_err


def calc_stats(runtimes):
    if len(runtimes) == 0:
        raise ValueError("runtimes array is empty")
    elif len(runtimes) == 1:
        runtime_mean = runtimes[0]
        runtime_err = 0.0
    else:
        runtime_mean = statistics.mean(runtimes)
        runtime_stdev = statistics.stdev(runtimes)

        # Filter out outliers
        runtimes_filtered = [runtime for runtime in runtimes if abs(runtime - runtime_mean) < 2 * runtime_stdev]
        if len(runtimes_filtered) < len(runtimes):
            runtime_mean = statistics.mean(runtimes_filtered)
            runtime_stdev = statistics.stdev(runtimes_filtered)

        runtime_err = 2 * runtime_stdev

    return runtime_mean, runtime_err

In [5]:
!make cleanall
!make FT CLASS=S
!make FT CLASS=W
!make FT CLASS=A
!make FT CLASS=B
!make FT CLASS=C
!make FT CLASS=D

rm -f core 
rm -f *~ */core */*~ */*.o */npbparams.hpp */*.obj */*.exe
rm -f sys/setparams sys/makesuite sys/setparams.hpp
rm -r bin/*
   =      NAS PARALLEL BENCHMARKS 4.1        =
   =      OpenMP Versions                    =
   =      C++                                =

cd FT; make CLASS=S
make[1]: Entering directory '/home/u216110/multi-core-processing-project/ours/FT'
make[2]: Entering directory '/home/u216110/multi-core-processing-project/ours/sys'
g++ -g -fopenmp -o setparams setparams.cpp
make[2]: Leaving directory '/home/u216110/multi-core-processing-project/ours/sys'
../sys/setparams ft S
make[1]: Warning: File 'npbparams.hpp' has modification time 0.00071 s in the future
g++ -std=c++14 -g -mavx512vl  -c -I../common -I/opt/intel/oneapi/vtune/2024.1/sdk/include -O3 -fopenmp -mcmodel=medium ft.cpp
cd ../common; g++ -std=c++14 -g -mavx512vl  -c -I../common -I/opt/intel/oneapi/vtune/2024.1/sdk/include -O3 -fopenmp -mcmodel=medium c_randdp.cpp
cd ../common; g++ -std=c++14 -g -m

In [ ]:
CLASSES = ['C', 'D']
# CLASSES = ['S', 'W', 'A', 'B', 'C', 'D']
num_iters=50

runtime_means, runtime_errs = execute_classes(CLASSES, num_iters)

print(f'runtime_means: {runtime_means}')
print(f'runtime_errs : {runtime_errs}')

In [6]:
class_ = 'A'
num_iters = 50

runtime_mean, runtime_err = execute_class(class_, num_iters)

print(f'runtime_mean: {runtime_mean}')
print(f'runtime_err : {runtime_err}')

# runtime_mean: 0.23564690000000002
# runtime_err : 0.04453268120730114

Class: A, Iterations: 50


  [001] Runtime: 0.239589 ± 0.000000 [sec] (0.239589)
  [002] Runtime: 0.225257 ± 0.040536 [sec] (0.210926)
  [003] Runtime: 0.222836 ± 0.029865 [sec] (0.217993)
  [004] Runtime: 0.246935 ± 0.099430 [sec] (0.319230)
  [005] Runtime: 0.242858 ± 0.088018 [sec] (0.226551)
  [006] Runtime: 0.238609 ± 0.081431 [sec] (0.217365)
  [007] Runtime: 0.224917 ± 0.023092 [sec] (0.237077)
  [008] Runtime: 0.248833 ± 0.090703 [sec] (0.321929)
  [009] Runtime: 0.248118 ± 0.084953 [sec] (0.242403)
  [010] Runtime: 0.256931 ± 0.097579 [sec] (0.336245)
  [011] Runtime: 0.258097 ± 0.092894 [sec] (0.269754)
  [012] Runtime: 0.254894 ± 0.091307 [sec] (0.219671)
  [013] Runtime: 0.255783 ± 0.087654 [sec] (0.266449)
  [014] Runtime: 0.253915 ± 0.085369 [sec] (0.229622)
  [015] Runtime: 0.255311 ± 0.082971 [sec] (0.274861)
  [016] Runtime: 0.250390 ± 0.070167 [sec] (0.262427)
  [017] Runtime: 0.247998 ± 0.070436 [sec] (0.212125)
  [018] Runtime: 0.246083 ± 0.070004 [sec] (0.215437)
  [019] Runtime: 0.246965 ± 

In [39]:
# Iterations:        1000           1000           200            100            50              2
ref_mean_by_class = {'S': 0.010051, 'W': 0.022201, 'A': 0.266957, 'B': 5.650252, 'C': 34.916714, 'D': 776.550364}
ref_err_by_class  = {'S': 0.001129, 'W': 0.001974, 'A': 0.114606, 'B': 0.178466, 'C':  0.538893, 'D':   2.278630}

class_ = 'A'
num_iters = 10

ref_mean = ref_mean_by_class[class_]
ref_err = ref_err_by_class[class_]

_ = benchmark_class(class_, num_iters, ref_mean, ref_err)

Class: A, Iterations: 10
  [001] Runtime: 0.035763 ± 0.000000 [sec] (0.035763)
  [002] Runtime: 0.036940 ± 0.003329 [sec] (0.038117)
  [003] Runtime: 0.036919 ± 0.002355 [sec] (0.036878)
  [004] Runtime: 0.037063 ± 0.002007 [sec] (0.037493)
  [005] Runtime: 0.037121 ± 0.001757 [sec] (0.037352)
  [006] Runtime: 0.037321 ± 0.001853 [sec] (0.038323)
  [007] Runtime: 0.037315 ± 0.001692 [sec] (0.037281)
  [008] Runtime: 0.037531 ± 0.001984 [sec] (0.039037)
  [009] Runtime: 0.037531 ± 0.001984 [sec] (0.031470)
  [010] Runtime: 0.036857 ± 0.004446 [sec] (0.021391)

Runtime: 0.036857 ± 0.004446 [sec]
Ref    : 0.476025 ± 0.004108 [sec]
Diff   : 0.439168 ± 0.006053 [sec]
Speedup:   92.26% ±    1.50%
